In [1]:
import re
import os
import pandas as pd
import numpy as np
import traceback2
from datetime import datetime
import pymysql
import numpy as np

where_map = {}
# pairs regexp: "\[([a-zA-z0-9$\-+τ\.%]*)\]-", "-\[([a-zA-z0-9$\-+τ]*)\]",  or ( c in ['τ'] for c in s)
ignorable_files = ["yieldFarm_7-9.csv", "Alternate Yields 7:9.xlsx", "farm_risk_yield_data_2021_07_09.csv", "unmapped_dex.csv", '.ipynb_checkpoints', "crash_parser.ipynb"]


In [2]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()

In [3]:
def get_raydium_data(date):
    sql = f'select * from raydium_farm_pool_data where timestamp_dt = FROM_UNIXTIME({date});'
    data = pd.read_sql_query(sql, __connect__())
    data['apr'] = data['pool_rate']
    data['network'] = 'SOL'
    data['type_of_agriculture'] = data['farm_payment'].apply(lambda x : 'FARM' if x is not None else 'POOL')
    return data[['pair1', 'pair2', 'apr', 'tvl', 'where', 'network', 'type_of_agriculture']]

def get_liquidityfolio_pool_data(date):
    sql = f'select * from liquidityfolio_pool where timestamp_dt = FROM_UNIXTIME({date}) and `where` != "balancer";'
    data = pd.read_sql_query(sql, __connect__())
    data['apr'] = data['pool_rate'] * 100 * 12
    data['tvl'] = data['tvl'] * 1000
    data['type_of_agriculture'] = data['farm_payment'].apply(lambda x : 'FARM' if x is not None else 'POOL')
    data['network'] = 'ETH'
    return data[['pair1', 'pair2', 'apr', 'tvl', 'where', 'network', 'type_of_agriculture']]

In [4]:
date = datetime(2021, 7, 12)
dt = date.timestamp()

In [5]:
df = get_raydium_data(dt)
df = df.append(get_liquidityfolio_pool_data(dt))
df[df['where'] == 'Sushi']

,pair1,pair2,apr,tvl,where,network,type_of_agriculture
0,ETH,BEL,155.122558,7.547389e+03,Sushi,ETH,POOL
1,ETH,UMA,3.767084,4.305553e+06,Sushi,ETH,POOL
2,ETH,UWL,0.264113,5.270969e+04,Sushi,ETH,POOL
4,stXYM,stETH,0.000000,0.000000e+00,Sushi,ETH,POOL
5,ETH,LIGHT,0.134295,1.960001e+03,Sushi,ETH,POOL
...,...,...,...,...,...,...,...
1803,AMIS,USDC,0.375174,9.974114e+01,Sushi,ETH,POOL
1804,ETH,SAK3,364.236329,1.856205e+03,Sushi,ETH,POOL
1809,ETH,89SA,0.000000,3.998762e+03,Sushi,ETH,POOL
1810,ETH,SEC,1.525689,6.920176e+04,Sushi,ETH,POOL


In [6]:
get_liquidityfolio_pool_data(dt)

,pair1,pair2,apr,tvl,where,network,type_of_agriculture
0,ETH,BEL,155.122558,7.547389e+03,Sushi,ETH,POOL
1,ETH,UMA,3.767084,4.305553e+06,Sushi,ETH,POOL
2,ETH,UWL,0.264113,5.270969e+04,Sushi,ETH,POOL
3,WBTC,USDC,5.161059,1.029026e+06,Uniswap,ETH,POOL
4,stXYM,stETH,0.000000,0.000000e+00,Sushi,ETH,POOL
...,...,...,...,...,...,...,...
1807,ETH,CHZ,8.838859,4.913366e+04,Uniswap,ETH,POOL
1808,ETH,POLS,3.309331,4.662414e+06,Uniswap,ETH,POOL
1809,ETH,89SA,0.000000,3.998762e+03,Sushi,ETH,POOL
1810,ETH,SEC,1.525689,6.920176e+04,Sushi,ETH,POOL


In [7]:
sm = {
    "WBTC" : "BTC",
    "WETH" : "ETH",
    "WBNB" : "BNB",
    "WFTM" : "FTM",
    "WAVAX" : "AVAX",
    "cvxCRV" : "CRV"
}


In [8]:
dex_map = {
    "SUSHI" : "https://app.sushi.com"
    
}

In [9]:
def get_alternate_pairs(line):
    spl = line.split("/")
    print(spl)
    if spl and len(spl) > 1:
        pair1 = spl[0].strip()
        print(pair1)
        pair2 = spl[1].strip()
        print(pair2)
        pair2 = pair2[0 : pair2.index(' ')]
        return [pair1, pair2]
    return None
    

In [10]:
def pair1(line):
#     print(line)
    try:
        is_3_pairs = line.split("]-[")
        if len(is_3_pairs) > 2:
            return None
        s_index = line.index("[") + 1
        s = line[s_index: line.index("]", s_index)]
        if all(ord(c) < 128 for c in s):
            return s
        else:
            print(line)
            return None
    except:
        return ''

def pair2(line):
#     print(line)
    s_index = line.index("-[") + 2
    s = line[s_index: line.index("]", s_index)]
#     print(s)
    if all(ord(c) < 128 for c in s):
        return s
    else:
        print(line)
        return ''

In [11]:
def apr(line):
    apr = "Year [0-9\,\.]+\%"
    x = re.findall(apr, line)
    return x[0].replace("Year ", "").replace("%", "").strip() if len(x) else 0

In [12]:
def tvl(line):
    tvl = "TVL: \$[0-9\.\,]+"
    x = re.findall(tvl, line)
    return x[0].replace("TVL: $", "").replace("%", "").replace(",","").strip()

In [13]:
def where(line):
    if 'INFO' in line:
        return line.replace("INFO", "").replace('\n', '').replace(":", "").replace("//", "://").replace('*', '').strip()
    return None

In [14]:
def get_single_pair(line): 
    exp = "\d* *.*Price:"
    x = re.findall(exp, line)
    if not len(x) or "TVL:" in line:
#         print(">>>>>>>>>>>",line)
        return
    pair = x[0].replace(" Price:", "")
    return pair[pair.rindex(" "):].strip() if " " in pair else pair

def get_single_tvl(line):
    exp = "\(*[0-9\,\.$]+\)*"
    x = re.findall(exp, line)
    return x[-1].replace("(", "").replace(")", "").replace("$", "").replace(",","").strip()


In [15]:
tvl("[WBTC]-[WAVAX] PGL [+] [-] [<=>] Price: $160,706,376.77 TVL: $12,218,373.64\n")

'12218373.64'

In [16]:
def is_end_of_block(line):
#     print(line, line == "" or len(re.findall("INFO[ ]+: ", line)))
    return line.strip() == "" or len(re.findall("INFO[ ]+: ", line)) or "******" in line

def read_block(lines, start_index):
    block = []
#     print("Block Start Line No.: ", start_index + 1)
    while start_index < len(lines):
        line = lines[start_index]
        start_index = start_index + 1
        if not is_end_of_block(line):
            block.append(line.strip())
        else:
            break
    return [block, start_index]

def read_blocks_from_files(file):
    f = open(file)
    lines = f.readlines()
    blocks = []
    block = []
    block_start_line_no = []
    line_no = 0
    start_index = 0
    while start_index < len(lines):
        line = lines[start_index]
        if not is_end_of_block(lines[start_index]):
            block_and_index = read_block(lines, start_index)
#             print(block_and_index[0])
            if block_and_index is not None:
                block_start_line_no.append(start_index + 1)
                start_index = block_and_index[1]
                blocks.append(block_and_index[0])
            else:
                start_index = start_index + 1
        elif len(re.findall("INFO[ ]+: ", line)):
            block_start_line_no.append(start_index + 1)
            blocks.append([line])
            start_index = start_index + 1
        else:
            start_index = start_index + 1
    return [blocks, block_start_line_no]

def read_blocks_from_files_as(file):
    f = open(file)
    lines = f.readlines()
    blocks = []
    block = []
    block_start_line_no = []
    line_no = 0
    for line in lines:
        line_no = line_no + 1
        line = line.strip()
        if line == "" or len(re.findall("INFO[ ]+: ", line)):
            blocks.append(block)
            block_start_line_no.append(line_no)
            if len(line):
                blocks.append([line])
                block_start_line_no.append(line_no + 1)
            block = []
        else:
            block.append(line)
            
    blocks.append(block)
    return [blocks, block_start_line_no]

def get_pair_info_from_block(block, wh, network):
    tvl_index = 0
    if network == 'BSC':
        tvl_index = 3
    if len(block) == 1:
        if 'INFO' in block[0]:
            return [where(block[0])]
        else:
            return [wh] if len(wh) else None
    if len(block) and pair1(block[0]) and not get_single_pair(block[0]):
        apr_index = -1
        for index in range(1, len(block)):
            if 'APR: ' in block[index]:
                apr_index = index
                break
        return [pair1(block[0]), pair2(block[0]), get_single_tvl(block[tvl_index]) if tvl_index > 0 else tvl(block[tvl_index]), apr(block[apr_index]) if apr_index > 1 else 0, where_map[wh] if wh in where_map else wh, network]
    elif len(block) > 2 and get_alternate_pairs(block[0]):
        print(get_alternate_pairs(block[0]))
        tvl_index = 0
        for index in range(1, len(block)):
            if 'Staked: ' in block[index]:
                tvl_index = index
                break
        apr_index = -1
        for index in range(1, len(block)):
            if 'APR: ' in block[index]:
                apr_index = index
                break
        pairs = get_alternate_pairs(block[0])
        return [pairs[0], pairs[1], get_single_tvl(block[tvl_index]), apr(block[apr_index]), where_map[wh] if wh in where_map else wh, network]
    elif len(block) > 2 and get_single_pair(block[0]):
#         print('singel')
        if network == 'BSC':
            tvl_index = 1
        apr_index = -1
        for index in range(1, len(block)):
            if 'APR: ' in block[index]:
                apr_index = index
                break
        tvl_index = 0
        if "TVL:" not in block[0]:
            for index in range(1, len(block)):
                if 'Stacked: ' in block[index]:
                    tvl_index = index
                    break
        return [get_single_pair(block[0]), "", get_single_tvl(block[tvl_index]), apr(block[apr_index]) if apr_index > 1 else 0, where_map[wh] if wh in where_map else wh, network]
    return None


def get_pairs(blocks, network, block_start_line_no):
    where = ""
    pairs = []
    index = -1
    for block in blocks:
#         print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        index = index + 1
        try:
            pair = get_pair_info_from_block(block, where, network)
            if pair == None:
                print('no Pair....\n',block)
                print("Block Start Line No.: ", block_start_line_no[index] if index < len(block_start_line_no) else -1)
#             if len(where) == 0 and pair and len(pair) > 2:
#                 print(block, network)
            if pair is not None and len(pair) > 0:
                if len(pair) == 1 and pair[0] and len(pair[0]):
                    where = pair[0]
                else:
                    pairs.append(pair)
        except Exception as e:
            print("Exception", traceback2.format_exc())
            print(block[0])
    return pairs
        

In [17]:
block = [['17 - mrenBTC Price: $33,923.83 TVL: $75,675.25', 'Staked: 2.2015 mrenBTC ($74,682.95)', 'MM Per Week: 58.02 ($193.79)', 'APR: Day 0.04% Week 0.26% Year 13.49%', 'You are staking 0.00 mrenBTC ($0.00), 0.00% of the pool.', 'Stake 0.00 mrenBTC', 'Unstake 0.00 mrenBTC', 'Claim 0.00 MM ($0.00)', 'Staking or unstaking also claims rewards.']]
get_pairs(block, "AVAX", [1])

['17 - mrenBTC Price: $33,923.83 TVL: $75,675.25']
no Pair....
 ['17 - mrenBTC Price: $33,923.83 TVL: $75,675.25', 'Staked: 2.2015 mrenBTC ($74,682.95)', 'MM Per Week: 58.02 ($193.79)', 'APR: Day 0.04% Week 0.26% Year 13.49%', 'You are staking 0.00 mrenBTC ($0.00), 0.00% of the pool.', 'Stake 0.00 mrenBTC', 'Unstake 0.00 mrenBTC', 'Claim 0.00 MM ($0.00)', 'Staking or unstaking also claims rewards.']
Block Start Line No.:  1


[]

In [18]:
# def get_single_tvl(line):
#     exp = "\(*[0-9\,\.$]+\)*"
#     x = re.findall(exp, line)
#     print(x)
#     return x[-1].replace("(", "").replace(")", "").replace("$", "").replace(",","").strip()

In [19]:
get_single_pair(block[0][0])

In [20]:
data_dir = './'
def conver_files_to_one_df(data_dir):
    all_pairs = []
    files = os.listdir(data_dir)
    for file_name in files:
        print(file_name)
        if file_name in ignorable_files:
            continue
        network = file_name[0:file_name.index(" ")]
        if 'Alternate' == network:
            network = 'ETH'
#         if 'MATIC' != network:
#             continue
        print(file_name)
        blocks = read_blocks_from_files(f"{data_dir}{file_name}")
        block_start_line_no = blocks[1]
        blocks = blocks[0]
        pairs = get_pairs(blocks, network, block_start_line_no)
        all_pairs.extend(pairs)
    return all_pairs

In [21]:
pairs = conver_files_to_one_df(data_dir)

ETH Yields 7:9.uu
ETH Yields 7:9.uu
no Pair....
 ['ETH Yields 7/9']
Block Start Line No.:  1
['DEX Price: $1.26 Market Cap: $529,200.00 [%]']
['KIF Price: $37.72 Market Cap: $1,546,519.74 [%]']
['kBASEv0 Price: $0.21 Market Cap: $37,142.37 [%]']
['Boardroom']
no Pair....
 ['Boardroom', 'There is a total 1619.93 BAS ($106.86) staked in the Boardroom.', 'You are staking 0 BAS ($0.00), 0.00% of the pool.', 'Stake 0.00 BAS', 'Unstake 0.00 BAS', 'Claim 0.00 BAC ($0.00)', 'Revoke (set approval to 0)', 'Exit']
Block Start Line No.:  502
['Boardroom']
no Pair....
 ['Boardroom', 'There is a total 30323.65 BSGS ($1,870.70) staked in the Boardroom.', 'You are staking 0 BSGS ($0.00), 0.00% of the pool.', 'Stake 0.0000 BSGS', 'Unstake 0.0000 BSGS', 'Claim 0.0000 BSG', 'Revoke (set approval to 0)', 'Exit']
Block Start Line No.:  546
['DAI Price: $1.00 Market Cap: $5,245,820,744.32 [%]']
['DPX Price: $119.86 Market Cap: $59,930,000.00 [%]']
['YFI Price: $33,426.00 Market Cap: $1,225,597,716.00 [%]']


no Pair....
 ['BSC Yields 7/9']
Block Start Line No.:  1
no Pair....
 ['Staking Contract', 'Found 429 pools.']
Block Start Line No.:  16
['0 - Cake Price: $14.45 Market Cap: $5,240,289,386.16 [%]']
['137 - LOTTO Price: $0.00 Market Cap: $0.00']
['138 - BURN Price: $0.00 Market Cap: $0.00']
361 - [τBTC]-[BTCB] Cake LP [+] [-] [<=>] Price: $1,336,978,328.10 TVL: $6,383,416.78
['361 - [τBTC]-[BTCB] Cake LP [+] [-] [<=>] Price: $1,336,978,328.10 TVL: $6,383,416.78']
no Pair....
 ['361 - [τBTC]-[BTCB] Cake LP [+] [-] [<=>] Price: $1,336,978,328.10 TVL: $6,383,416.78', 'τBTC Price: $13,094.70', 'BTCB Price: $33,668.00', 'Staked: 0.0047 Cake-LP ($6,310,323.00)', 'CAKE Per Week: 4976.18 ($71,905.81)', 'APR: Day 0.16% Week 1.14% Year 59.25%', 'You are staking 0.00 [τBTC]-[BTCB] Cake LP ($0.00), 0.00% of the pool.', 'Stake 0.00 [τBTC]-[BTCB] Cake LP', 'Unstake 0.00 [τBTC]-[BTCB] Cake LP', 'Claim 0.00 CAKE ($0.00)', 'Staking or unstaking also claims rewards.']
Block Start Line No.:  1169
380 - [W

45 - stkBNB
ADAv2 Price: $0.00 Market Cap: $0.00
['45 - stkBNB', 'ADAv2 Price: $0.00 Market Cap: $0.00']
45 - stkBNB
ADAv2 Price: $0.00 Market Cap: $0.00
['45 - stkBNB', 'ADAv2']
['45 - stkBNB', 'ADAv2 Price: $0.00 Market Cap: $0.00']
45 - stkBNB
ADAv2 Price: $0.00 Market Cap: $0.00
['46 - stkBUSD', 'USTv2 Price: $0.00 Market Cap: $0.00']
46 - stkBUSD
USTv2 Price: $0.00 Market Cap: $0.00
['46 - stkBUSD', 'USTv2 Price: $0.00 Market Cap: $0.00']
46 - stkBUSD
USTv2 Price: $0.00 Market Cap: $0.00
['46 - stkBUSD', 'USTv2']
['46 - stkBUSD', 'USTv2 Price: $0.00 Market Cap: $0.00']
46 - stkBUSD
USTv2 Price: $0.00 Market Cap: $0.00
['47 - stkBUSD', 'BTCBv2 Price: $0.00 Market Cap: $0.00']
47 - stkBUSD
BTCBv2 Price: $0.00 Market Cap: $0.00
['47 - stkBUSD', 'BTCBv2 Price: $0.00 Market Cap: $0.00']
47 - stkBUSD
BTCBv2 Price: $0.00 Market Cap: $0.00
['47 - stkBUSD', 'BTCBv2']
['47 - stkBUSD', 'BTCBv2 Price: $0.00 Market Cap: $0.00']
47 - stkBUSD
BTCBv2 Price: $0.00 Market Cap: $0.00
['48 - stkbeltB

['nrvRC_ETH_BUSD Price: $1.00 Market Cap: $335,602.66 [%]']
['nrvRC_NRV_BUSD Price: $1.00 Market Cap: $524,415.98 [%]']
['11 - WBNB Price: $315.31 Market Cap: $2,239,385,302.27 [%]']
['12 - BUSD Price: $1.00 Market Cap: $4,201,000,000.00 [%]']
['13 - ETH Price: $2,163.79 Market Cap: $2,823,745,950.00 [%]']
['14 - BTCB Price: $33,677.00 Market Cap: $2,711,032,177.00 [%]']
['15 - Cake Price: $14.33 Market Cap: $5,197,567,370.72 [%]']
['16 - USDT Price: $1.00 Market Cap: $3,179,997,922.18 [%]']
['17 - DAI Price: $1.00 Market Cap: $262,000,000.00 [%]']
['0 - TOMAHAWK Price: $0.00 Market Cap: $0.00']
['13 - BUSD Price: $1.00 Market Cap: $4,201,000,000.00 [%]']
['14 - WBNB Price: $314.94 Market Cap: $2,236,714,434.65 [%]']
['15 - USDT Price: $1.00 Market Cap: $3,179,997,922.18 [%]']
['16 - BTCB Price: $33,568.00 Market Cap: $2,702,257,568.00 [%]']
['17 - USDC Price: $1.00 Market Cap: $1,779,000,000.00 [%]']
['18 - Cake Price: $14.29 Market Cap: $5,183,136,916.52 [%]']
['19 - DAI Price: $1.00

['6 - DAI Price: $1.00 Market Cap: $420,455,935.63 [%]']
['7 - USDC Price: $1.00 Market Cap: $1,111,466,160.34 [%]']
['8 - WBTC Price: $34,199.00 Market Cap: $613,106,954.82 [%]']
['9 - WETH Price: $2,140.56 Market Cap: $1,636,688,015.97 [%]']
['10 - QUICK Price: $441.42 Market Cap: $430,207,212.96 [%]']
['11 - AAVE Price: $303.17 Market Cap: $144,023,114.28 [%]']
['12 - SUSHI Price: $8.12 Market Cap: $11,315,535.97 [%]']
['13 - LINK Price: $18.38 Market Cap: $58,875,390.78 [%]']
['14 - FISH Price: $0.00 Market Cap: $0.00']
['12 - UNI-V2 Price: $0.00 Market Cap: $0.00']
['14 - UNI-V2 Price: $0.00 Market Cap: $0.00']
['15 - UNI-V2 Price: $0.00 Market Cap: $0.00']
['24 - QUICK Price: $441.42 Market Cap: $430,207,212.96 [%]']
['26 - QUICK Price: $441.42 Market Cap: $430,207,212.96 [%]']
['27 - QUICK Price: $441.42 Market Cap: $430,207,212.96 [%]']
['28 - WETH Price: $2,140.56 Market Cap: $1,636,688,015.97 [%]']
['30 - WETH Price: $2,140.56 Market Cap: $1,636,688,015.97 [%]']
['31 - WETH P

In [22]:
pair_df = pd.DataFrame(pairs, columns=['pair1', 'pair2', 'tvl', 'apr', 'where', 'network'])
pair_df['apy'] = np.nan

In [23]:
pairs

[['ETH',
  '1INCH',
  '9289156.66',
  '0.00',
  'https://1inch.exchange/#/dao/farming',
  'ETH'],
 ['1INCH',
  'VSP',
  '3543789.18',
  '0.00',
  'https://1inch.exchange/#/dao/farming',
  'ETH'],
 ['ETH',
  'TORN',
  '361611.45',
  '0.00',
  'https://1inch.exchange/#/dao/farming',
  'ETH'],
 ['ETH',
  'OPIUM',
  '1429822.08',
  '0.00',
  'https://1inch.exchange/#/dao/farming',
  'ETH'],
 ['LDO',
  'stETH',
  '3805987.90',
  '0.00',
  'https://1inch.exchange/#/dao/farming',
  'ETH'],
 ['ETH',
  'WBTC',
  '1027517.94',
  '0.00',
  'https://1inch.exchange/#/dao/farming',
  'ETH'],
 ['ETH',
  'USDC',
  '636734.81',
  '0.00',
  'https://1inch.exchange/#/dao/farming',
  'ETH'],
 ['ETH',
  'DAI',
  '256859.19',
  '0.00',
  'https://1inch.exchange/#/dao/farming',
  'ETH'],
 ['ETH',
  'USDT',
  '376418.92',
  '0.00',
  'https://1inch.exchange/#/dao/farming',
  'ETH'],
 ['DEX', 'WETH', '96957.25', '8.98', 'https://kitten.finance/', 'ETH'],
 ['DEX', '', '529200.00', '7.99', 'https://kitten.financ

In [24]:
pair_df

,pair1,pair2,tvl,apr,where,network,apy
0,ETH,1INCH,9289156.66,0.00,https://1inch.exchange/#/dao/farming,ETH,NaN
1,1INCH,VSP,3543789.18,0.00,https://1inch.exchange/#/dao/farming,ETH,NaN
2,ETH,TORN,361611.45,0.00,https://1inch.exchange/#/dao/farming,ETH,NaN
3,ETH,OPIUM,1429822.08,0.00,https://1inch.exchange/#/dao/farming,ETH,NaN
4,LDO,stETH,3805987.90,0.00,https://1inch.exchange/#/dao/farming,ETH,NaN
...,...,...,...,...,...,...,...
8715,WFTM,BTC,3431218.82,260.83,https://fantom.bullrun.finance,FANTOM,NaN
8716,SBRW,,104112.08,366.89,https://www.strangebrew.finance,FANTOM,NaN
8717,WFTM,SBRW,2077.87,437.19,https://www.strangebrew.finance,FANTOM,NaN
8718,USDC,SBRW,9508.48,298.62,https://www.strangebrew.finance,FANTOM,NaN


In [25]:
pair_df = pair_df.drop(pair_df[pair_df['pair1'] == ''].index)

In [26]:
pair_df[pair_df['pair1'] == '']

,pair1,pair2,tvl,apr,where,network,apy


In [27]:
pair_df[pair_df['network'] == 'Alternate']['network'] = 'ETH'

In [28]:
sheet_pairs_df = pd.DataFrame([])
eth_sheet_pairs_df = pd.read_csv('yieldFarm_7-9.csv')
# eth_sheet_pairs_df.columns=['pair1', 'pair2', 'tvl', 'where', 'apr']
eth_sheet_pairs_df['apy'] = np.nan
eth_sheet_pairs_df['network'] = 'ETH'
sheet_pairs_df = sheet_pairs_df.append(eth_sheet_pairs_df)
sheet_pairs_df['pair2'] = sheet_pairs_df['pair2'].fillna('')

In [29]:
sheet_pairs_df

,pair1,pair2,apr,tvl,where,network,apy
0,WETH,USDT,12.81,258536006.3,https://app.sushi.com/farms,ETH,NaN
1,USDC,WETH,11.56,362487636.3,https://app.sushi.com/farms,ETH,NaN
2,DAI,WETH,14.18,224085916.7,https://app.sushi.com/farms,ETH,NaN
3,sUSD,WETH,17.01,10331566.09,https://app.sushi.com/farms,ETH,NaN
4,COMP,WETH,7.96,27158281.1,https://app.sushi.com/farms,ETH,NaN
...,...,...,...,...,...,...,...
249,WMATIC,DAI,43.81,4260570.37,https://dmm.exchange/,ETH,NaN
250,WMATIC,KNC,96.65,2928140.2,https://dmm.exchange/,ETH,NaN
251,USDC,USDT,12.16,45427408.59,https://dmm.exchange/,ETH,NaN
252,USDC,DAI,16.49,33802024.65,https://dmm.exchange/,ETH,NaN


In [30]:
def update_apr(apr):
    apr = f"{apr}".replace("%", "")
    if len(apr) == 0 or "nan" == apr:
        return np.nan
    else:
        return apr

In [31]:
def get_symbol_mapping(pair):
    if pair is not None:
        return sm[pair.strip()] if pair.strip() in sm else pair.strip()
    else:
        return ""

In [32]:
pair_df = pair_df.append(sheet_pairs_df).reset_index(drop=True)
pair_df['type_of_agriculture'] = 'FARM'
pair_df = pair_df.append(df).reset_index(drop=True)

In [33]:

pair_df['apr'] = pair_df['apr'].apply(lambda x: update_apr(x))
pair_df['pair1'] = pair_df['pair1'].apply(lambda x: sm[x.strip()] if x.strip() in sm else x.strip())
pair_df['pair2'] = pair_df['pair2'].apply(lambda x: get_symbol_mapping(x))

In [34]:
pair_df['pair2'].unique()

array(['1INCH', 'VSP', 'TORN', ..., '89SA', 'SEC', 'OHM'], dtype=object)

In [35]:
pair_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,ETH,1INCH,9289156.66,0.00,https://1inch.exchange/#/dao/farming,ETH,NaN,FARM
1,1INCH,VSP,3543789.18,0.00,https://1inch.exchange/#/dao/farming,ETH,NaN,FARM
2,ETH,TORN,361611.45,0.00,https://1inch.exchange/#/dao/farming,ETH,NaN,FARM
3,ETH,OPIUM,1429822.08,0.00,https://1inch.exchange/#/dao/farming,ETH,NaN,FARM
4,LDO,stETH,3805987.90,0.00,https://1inch.exchange/#/dao/farming,ETH,NaN,FARM
...,...,...,...,...,...,...,...,...
10887,ETH,CHZ,49133.657339,8.8388592,Uniswap,ETH,NaN,POOL
10888,ETH,POLS,4662413.928989,3.30933108,Uniswap,ETH,NaN,POOL
10889,ETH,89SA,3998.762313,0.0,Sushi,ETH,NaN,POOL
10890,ETH,SEC,69201.760211,1.5256885199999999,Sushi,ETH,NaN,POOL


In [36]:
# pair_df['apr'].unique().tolist()

In [37]:
def convert_apr_to_apy(pair_df):
    tmp_df = pair_df[pair_df['apy'].isna()]
    print(tmp_df.shape)
    for index, row in tmp_df.iterrows():
        apr = row['apr']
        if pd.isnull( apr):
            return
        if apr is not None and len(apr.strip()):
            apr = float(apr.replace(',',''))
        else:
            apr = 0
        try:
            pair_df.loc[index, 'apy'] = ((1+(apr/100)/365)**365-1)*100
        except:
            pair_df.loc[index, 'apy'] = np.nan
    return
            
def convert_apy_to_apr(pair_df):
    tmp_df = pair_df[(pair_df['apr'].isnull())]
    print(tmp_df.shape)
    for index, row in tmp_df.iterrows():
        apy = row['apy']
        if pd.isnull( apy):
            return
        if apy is not None and len(apy.strip()):
            apy = float(apy.replace(',',''))
        else:
            apy = 0
        try:
            pair_df.loc[index, 'apr'] = (((1+(apy/100))**(1/365)-1)*365)*100
        except:
            pair_df.loc[index, 'apr'] = np.nan
    return


convert_apr_to_apy(pair_df)
convert_apy_to_apr(pair_df)

(10892, 8)
(54, 8)


In [38]:
pair_df[pair_df['apy'].isna()]

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
243,USDC,,25242101183.80,2298430.04,https://app.sirenmarkets.com/stake,ETH,NaN,FARM
933,BNB,,2196512494.19,6692443338563333120.00,https://autofarm.network,BSC,NaN,FARM
3184,ETH,BNB,0.00,436802050.24,https://www.gengarfinance.com,BSC,NaN,FARM
3193,ETH,,2840384700.00,346249356.11,https://www.gengarfinance.com,BSC,NaN,FARM
4096,ETH,BNB,0.00,2193284.41,https://champagne.farm,BSC,NaN,FARM
...,...,...,...,...,...,...,...,...
10887,ETH,CHZ,49133.657339,8.8388592,Uniswap,ETH,NaN,POOL
10888,ETH,POLS,4662413.928989,3.30933108,Uniswap,ETH,NaN,POOL
10889,ETH,89SA,3998.762313,0.0,Sushi,ETH,NaN,POOL
10890,ETH,SEC,69201.760211,1.5256885199999999,Sushi,ETH,NaN,POOL


In [39]:
pair_df[pair_df['apr'].isna()]

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
8746,REDUCE,,0,NaN,https://app.sushi.com/farms,ETH,NaN,FARM
8788,MITHCHEF,,0,NaN,https://app.sushi.com/farms,ETH,NaN,FARM
8836,kmWETH/USDT,LINK,0,NaN,https://app.sushi.com/farms,ETH,NaN,FARM
8837,kmWETH/USDC,LINK,0,NaN,https://app.sushi.com/farms,ETH,NaN,FARM
8838,kmWETH/DAI,LINK,0,NaN,https://app.sushi.com/farms,ETH,NaN,FARM
8839,kmWBTC/USDT,LINK,0,NaN,https://app.sushi.com/farms,ETH,NaN,FARM
8840,kmWBTC/DAI,LINK,0,NaN,https://app.sushi.com/farms,ETH,NaN,FARM
8841,kmWBTC/USDC,LINK,0,NaN,https://app.sushi.com/farms,ETH,NaN,FARM
8842,kmLINK/USDT,LINK,0,NaN,https://app.sushi.com/farms,ETH,NaN,FARM
8843,kmLINK/DAI,LINK,0,NaN,https://app.sushi.com/farms,ETH,NaN,FARM


In [40]:
pair_df = pair_df.drop_duplicates()

In [41]:
single_piar_df = pair_df[pair_df['pair2'] == ""].copy()
double_coin_pair_df = pair_df[pair_df['pair2'] != ""].copy()
tmp_df = double_coin_pair_df.copy()

In [42]:
single_piar_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
10,DEX,,529200.00,7.99,https://kitten.finance/,ETH,8.316927,FARM
11,KIF,,1546519.74,0.89,https://kitten.finance/,ETH,0.893961,FARM
14,kBASEv0,,37142.37,19.57,https://kitten.finance/,ETH,21.609822,FARM
38,DAI,,5245820744.32,0.00,https://debaseonomics.io/,ETH,0.000000,FARM
42,DPX,,59930000.00,16.07,https://app.dopex.io/farms,ETH,17.429109,FARM
...,...,...,...,...,...,...,...,...
8968,50C,,"41,908",1604.95,https://50cent.network,ETH,NaN,FARM
9570,Xi,,0.0,0.0,Sushi,ETH,NaN,POOL
9652,RUNE,,29.239175,0.45581604,Sushi,ETH,NaN,POOL
10234,SUSHI,,0.0,0.0,Sushi,ETH,NaN,POOL


In [43]:
double_coin_pair_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,ETH,1INCH,9289156.66,0.00,https://1inch.exchange/#/dao/farming,ETH,0.0,FARM
1,1INCH,VSP,3543789.18,0.00,https://1inch.exchange/#/dao/farming,ETH,0.0,FARM
2,ETH,TORN,361611.45,0.00,https://1inch.exchange/#/dao/farming,ETH,0.0,FARM
3,ETH,OPIUM,1429822.08,0.00,https://1inch.exchange/#/dao/farming,ETH,0.0,FARM
4,LDO,stETH,3805987.90,0.00,https://1inch.exchange/#/dao/farming,ETH,0.0,FARM
...,...,...,...,...,...,...,...,...
10887,ETH,CHZ,49133.657339,8.8388592,Uniswap,ETH,NaN,POOL
10888,ETH,POLS,4662413.928989,3.30933108,Uniswap,ETH,NaN,POOL
10889,ETH,89SA,3998.762313,0.0,Sushi,ETH,NaN,POOL
10890,ETH,SEC,69201.760211,1.5256885199999999,Sushi,ETH,NaN,POOL


In [44]:
tmp_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,ETH,1INCH,9289156.66,0.00,https://1inch.exchange/#/dao/farming,ETH,0.0,FARM
1,1INCH,VSP,3543789.18,0.00,https://1inch.exchange/#/dao/farming,ETH,0.0,FARM
2,ETH,TORN,361611.45,0.00,https://1inch.exchange/#/dao/farming,ETH,0.0,FARM
3,ETH,OPIUM,1429822.08,0.00,https://1inch.exchange/#/dao/farming,ETH,0.0,FARM
4,LDO,stETH,3805987.90,0.00,https://1inch.exchange/#/dao/farming,ETH,0.0,FARM
...,...,...,...,...,...,...,...,...
10887,ETH,CHZ,49133.657339,8.8388592,Uniswap,ETH,NaN,POOL
10888,ETH,POLS,4662413.928989,3.30933108,Uniswap,ETH,NaN,POOL
10889,ETH,89SA,3998.762313,0.0,Sushi,ETH,NaN,POOL
10890,ETH,SEC,69201.760211,1.5256885199999999,Sushi,ETH,NaN,POOL


In [45]:
tmp_df['tmp'] = tmp_df['pair1']
tmp_df['pair1'] = tmp_df['pair2']
tmp_df['pair2'] = tmp_df['tmp']
del tmp_df['tmp']

In [46]:
double_coin_pair_df.append(tmp_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,,sil,0.0,0.0,Sushi,ETH,NaN,POOL
1,$BASED,ETH,8171.793935,435.78845496,Uniswap,ETH,NaN,POOL
2,$BASED,SUSD,9102.133705,12.30405252,Uniswap,ETH,NaN,POOL
3,$BASED,sUSD,95.290903,0.0,Sushi,ETH,NaN,POOL
4,$DG,BNB,2052889.12,182.14,https://pancakeswap.finance,BSC,515.257408,FARM
...,...,...,...,...,...,...,...,...
14727,zLOT,ETH,18555.303736,1.1876888399999999,Sushi,ETH,NaN,POOL
14728,zSUSHI,ZERO,30111.96,0.00,https://app.0.exchange,AVAX,0.000000,FARM
14729,zUNI,ZERO,10249.69,0.00,https://app.0.exchange,AVAX,0.000000,FARM
14730,zUSDC,ZERO,20355.33,0.00,https://app.0.exchange,AVAX,0.000000,FARM


In [47]:
summary = {
    "total_pairs": pair_df.shape[0],
"single_coin_pairs": single_piar_df.shape[0],
"double_coin_piars": double_coin_pair_df.shape[0],
"after_switching_coins_total_double_coin_pairs": double_coin_pair_df.shape[0] * 2,
"total_pairs_in_sheet": double_coin_pair_df.shape[0] * 2 + single_piar_df.shape[0]
}

In [48]:
summary

{'total_pairs': 10580,
 'single_coin_pairs': 3214,
 'double_coin_piars': 7366,
 'after_switching_coins_total_double_coin_pairs': 14732,
 'total_pairs_in_sheet': 17946}

In [49]:
pair_df = double_coin_pair_df.append(tmp_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)
pair_df = single_piar_df.append(pair_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)

In [50]:
pair_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,,sil,0.0,0.0,Sushi,ETH,NaN,POOL
1,$BASED,ETH,8171.793935,435.78845496,Uniswap,ETH,NaN,POOL
2,$BASED,SUSD,9102.133705,12.30405252,Uniswap,ETH,NaN,POOL
3,$BASED,sUSD,95.290903,0.0,Sushi,ETH,NaN,POOL
4,$DG,BNB,2052889.12,182.14,https://pancakeswap.finance,BSC,515.257408,FARM
...,...,...,...,...,...,...,...,...
17941,zs-XUSD2,,0.00,0,https://www.stabilize.finance/app/,ETH,0.000000,FARM
17942,zs-tsBTC,,0.00,0,https://www.stabilize.finance/app/,ETH,0.000000,FARM
17943,τBTC,,23285828.98,383.18,https://www.elixir-swap.io,BSC,4423.297664,FARM
17944,🌾,,9469.12,327.43,https://harvester.app,MATIC,2504.174657,FARM


In [51]:
pair_df['dex'] = pair_df['where']
pair_df['date'] = date 
del pair_df['where']

In [52]:
# pair_df.to_csv('yield_data_2021_06_21.csv')

In [53]:
pair_df['dex'].unique().tolist()

['Sushi',
 'Uniswap',
 'https://pancakeswap.finance',
 'https://quickswap.exchange/#/quick',
 'https://strudel.finance',
 'https://swap.cometh.io/#/stake',
 'https://polygon.cosmicswap.finance',
 'https://gemstones.finance',
 'https://absorber.finance',
 'https://icecreamswap.finance',
 'http://mangofarm.finance',
 'https://brussels.worldswap.finance',
 'https://berlin.worldswap.finance',
 'https://1inch.exchange/#/dao/farming',
 'https://app.sushi.com/farms',
 'https://wheat.growthdefi.com',
 'https://www.convexfinance.com',
 'https://app.swamp.finance',
 'https://ellipsis.finance',
 'https://app.firebird.finance',
 'https://frozenyogurt.finance',
 'https://nerve.fi',
 'https://autofarm.network',
 'https://beta.belt.fi',
 'https://blue.planetfinance.io',
 'https://app.yieldparrot.finance',
 'https://50cent.network',
 'https://glacierswap.org/farms/',
 'https://8ballfinance.io',
 'https://dollar.supra.finance',
 'https://www.bigdataprotocol.com/datavault',
 'https://polyvolve.finance',

In [54]:
pair_df[pair_df['pair1'] == 'ETH']

,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date
7780,ETH,,14469896620.73,0,ETH,0.000000,FARM,https://www.bigdataprotocol.com/datavault,2021-07-12
7781,ETH,,14470055939.11,0.00,ETH,0.000000,FARM,https://app.thisistheway.finance/#/stake,2021-07-12
7782,ETH,,14470074182.36,0.00,ETH,0.000000,FARM,https://app.cryptex.finance,2021-07-12
7783,ETH,,2848514850.00,174.00,BSC,467.383773,FARM,https://donutdefi.com,2021-07-12
7784,ETH,,2848514850.00,0.00,BSC,0.000000,FARM,https://www.goosedefi.com,2021-07-12
...,...,...,...,...,...,...,...,...,...
9993,ETH,yyDAI+y...,119762.16423,1.76829936,ETH,NaN,POOL,Uniswap,2021-07-12
9994,ETH,zHEGIC,2026.173877,0.97085076,ETH,NaN,POOL,Uniswap,2021-07-12
9995,ETH,zHEGIC,16.541198,0.0,ETH,NaN,POOL,Sushi,2021-07-12
9996,ETH,zLOT,11.083493,1.3330256399999998,ETH,NaN,POOL,Uniswap,2021-07-12


In [55]:
pair_df['apr'].unique().tolist()

['0.0',
 '435.78845496',
 '12.30405252',
 '182.14',
 '63.92',
 '12.6326634',
 '22.145634360000003',
 '67.42',
 '1.19721756',
 '4.00070304',
 '143.96',
 '3.5534354400000003',
 '3.6866379600000005',
 '0.00',
 '0',
 '6.81569676',
 '3.23270472',
 '0.9484544399999999',
 '0.25865736',
 '275.23',
 '668.92',
 '10.12',
 '15.4448166',
 '9.77489292',
 '0.54230472',
 '0.29611872',
 '0.08252688',
 '13.592718',
 '0.02613648',
 '98.14',
 '0.57514584',
 '1.33073124',
 '0.6977059200000001',
 '0.9396170399999999',
 '0.0041978399999999996',
 '16.14',
 '4.16',
 '13.54',
 '6.59',
 '19.10',
 '4.63',
 '1604.95',
 '324382.94',
 '3160.41',
 '5516.15',
 '0.14172888',
 '892.67',
 '3.33547368',
 '13.33201872',
 '511.69',
 '1740.71',
 '1237.64',
 '2.43',
 '0.12039972',
 '64.13',
 '279.32',
 '74.16',
 '290.93',
 '15.28',
 '429.89',
 '51.65',
 '93.71',
 '91.28',
 '274.49',
 '16654.51',
 '12.89',
 '19.83',
 '109.98',
 '941.81',
 '101.46',
 '125.02',
 '375.07',
 '1000.19',
 '62.29',
 '7.57',
 '11.71',
 '52.75',
 '86.1

In [56]:
pair_df

,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date
0,,sil,0.0,0.0,ETH,NaN,POOL,Sushi,2021-07-12
1,$BASED,ETH,8171.793935,435.78845496,ETH,NaN,POOL,Uniswap,2021-07-12
2,$BASED,SUSD,9102.133705,12.30405252,ETH,NaN,POOL,Uniswap,2021-07-12
3,$BASED,sUSD,95.290903,0.0,ETH,NaN,POOL,Sushi,2021-07-12
4,$DG,BNB,2052889.12,182.14,BSC,515.257408,FARM,https://pancakeswap.finance,2021-07-12
...,...,...,...,...,...,...,...,...,...
17941,zs-XUSD2,,0.00,0,ETH,0.000000,FARM,https://www.stabilize.finance/app/,2021-07-12
17942,zs-tsBTC,,0.00,0,ETH,0.000000,FARM,https://www.stabilize.finance/app/,2021-07-12
17943,τBTC,,23285828.98,383.18,BSC,4423.297664,FARM,https://www.elixir-swap.io,2021-07-12
17944,🌾,,9469.12,327.43,MATIC,2504.174657,FARM,https://harvester.app,2021-07-12


In [57]:
pair_df[pair_df['pair1'] == 'CRV']

,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date
5992,CRV,,2716020239.34,0.00,ETH,0.000000,FARM,https://gamestop.finance,2021-07-12
5993,CRV,,101049000.20,31.72,ETH,37.308804,FARM,https://www.convexfinance.com,2021-07-12
5994,CRV,,4324918.78,22.67,MATIC,25.436520,FARM,https://harvester.app,2021-07-12
5995,CRV,,4349492.18,0.00,MATIC,0.000000,FARM,https://www.chickenfarms.live,2021-07-12
5996,CRV,,1647075.12,52.18,FANTOM,68.443025,FARM,https://klingftm.borgswap.exchange,2021-07-12
5997,CRV,,20424412,3.81,ETH,NaN,FARM,https://aave.com,2021-07-12
5998,CRV,CRV,24655977.10,52.39,ETH,68.796620,FARM,https://www.convexfinance.com,2021-07-12
5999,CRV,CRV,11705774.766457,4.685000759999999,ETH,NaN,POOL,Sushi,2021-07-12
6000,CRV,CRV,24655977.10,52.39,ETH,68.796620,FARM,https://www.convexfinance.com,2021-07-12
6001,CRV,CRV,11705774.766457,4.685000759999999,ETH,NaN,POOL,Sushi,2021-07-12


In [58]:
pair_df = pair_df.drop_duplicates()

## Farm Risk

In [59]:
farm_grades = pd.read_csv('~/notebooks/vfat_tool_data/Yield_grades.csv')
farm_grades

,Farm,Website,Network,RugDoc / Indv.,Grade
0,Aave,https://aave.com,ETH,NaN,A
1,Armor,https://armor.fi,ETH,NaN,A
2,UniSwap (Pool),uniswap.org,ETH,NaN,A
3,Aave,https://aave.com,MATIC,NaN,A
4,Sushi,https://app.sushi.com,MATIC,NaN,A
...,...,...,...,...,...
761,NaN,https://app.xsigma.fi,ETH,NaN,C
762,NaN,https://app.pickle.finance/,MATIC,NaN,D
763,NaN,https://polyfunky.com/,MATIC,NaN,F
764,NaN,https://www.helmet.insure,ETH,NaN,C


In [60]:
farm_grades = farm_grades[['Website', 'Network', 'Grade']]
farm_grades = farm_grades.append({'Website': 'Sushi', 'Network': 'ETH', 'Grade': 'A'}, ignore_index=True)
farm_grades = farm_grades.append({'Website': 'https://app.sushi.com/farms/', 'Network': 'ETH', 'Grade': 'A'}, ignore_index=True)
farm_grades = farm_grades.append({'Website': 'Uniswap', 'Network': 'ETH', 'Grade': 'A'}, ignore_index=True)

In [61]:
farm_grades

,Website,Network,Grade
0,https://aave.com,ETH,A
1,https://armor.fi,ETH,A
2,uniswap.org,ETH,A
3,https://aave.com,MATIC,A
4,https://app.sushi.com,MATIC,A
...,...,...,...
764,https://www.helmet.insure,ETH,C
765,https://app.pendle.finance/,ETH,D
766,Sushi,ETH,A
767,https://app.sushi.com/farms/,ETH,A


In [62]:
pairs_farm_risk_df = pd.merge(pair_df, farm_grades, how='left', left_on=['dex', 'network'], right_on=['Website', 'Network'])
pairs_farm_risk_df.to_csv('farm_risk_yield_data_2021_07_09.csv')
pairs_farm_risk_df


,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date,Website,Network,Grade
0,,sil,0.0,0.0,ETH,NaN,POOL,Sushi,2021-07-12,Sushi,ETH,A
1,$BASED,ETH,8171.793935,435.78845496,ETH,NaN,POOL,Uniswap,2021-07-12,Uniswap,ETH,A
2,$BASED,SUSD,9102.133705,12.30405252,ETH,NaN,POOL,Uniswap,2021-07-12,Uniswap,ETH,A
3,$BASED,sUSD,95.290903,0.0,ETH,NaN,POOL,Sushi,2021-07-12,Sushi,ETH,A
4,$DG,BNB,2052889.12,182.14,BSC,515.257408,FARM,https://pancakeswap.finance,2021-07-12,https://pancakeswap.finance,BSC,B
...,...,...,...,...,...,...,...,...,...,...,...,...
17996,zs-XUSD2,,0.00,0,ETH,0.000000,FARM,https://www.stabilize.finance/app/,2021-07-12,https://www.stabilize.finance/app/,ETH,D
17997,zs-tsBTC,,0.00,0,ETH,0.000000,FARM,https://www.stabilize.finance/app/,2021-07-12,https://www.stabilize.finance/app/,ETH,D
17998,τBTC,,23285828.98,383.18,BSC,4423.297664,FARM,https://www.elixir-swap.io,2021-07-12,https://www.elixir-swap.io,BSC,F
17999,🌾,,9469.12,327.43,MATIC,2504.174657,FARM,https://harvester.app,2021-07-12,https://harvester.app,MATIC,D


In [63]:
pairs_farm_risk_df[(pairs_farm_risk_df['Website'].isna()) & (pairs_farm_risk_df['type_of_agriculture'] == 'POOL')]

,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date,Website,Network,Grade


In [64]:
unmapped_dexs_df = pairs_farm_risk_df[pairs_farm_risk_df['Website'].isna()][['dex', 'network']].reset_index(drop=True)
unmapped_dexs_df = unmapped_dexs_df.drop_duplicates()
if unmapped_dexs_df.shape[0] > 1:
    unmapped_dexs_df.to_csv('unmapped_dex.csv')
unmapped_dexs_df


,dex,network
0,https://farm.boneswap.finance,MATIC
1,https://bisonfinance.online,MATIC
2,https://polymocha.finance,MATIC
3,https://www.polyowl.finance,MATIC
4,https://vampirefarming.club,MATIC
5,https://www.polygarden.finance,MATIC
6,https://dojofarm.finance,MATIC
7,https://www.polymaple.com,MATIC
8,https://polytopia.farm,MATIC
9,https://www.chickenfarms.live,MATIC
